In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/module/HeadPoseEstimator')

In [7]:
%%writefile Headpose.py


import os
import cv2
import sys
import numpy as np
from math import cos, sin
from moviepy.editor import *
from moviepy.editor import *
from keras import backend as K
import argparse
import matplotlib.pyplot as plt
from HeadPoseEstimator.face_direction.FSANET_model import FSA_net_Capsule, FSA_net_Var_Capsule,FSA_net_noS_Capsule
from HeadPoseEstimator.face_direction.FSANET_face_direction import *
from mtcnn.mtcnn import MTCNN
from keras.layers import Input,Average
from keras.models import Model
import time
from config import *

class HeadposeEstimation:

  def __init__(self,fsanet_model_path,option=None):

      if option is None:

         self.option = {
              'image_size' : 64,
              'num_capsule' : 3,
              'dim_capsule' : 16,
              'routings' : 2,
              'stage_num' : [3,3,3],
              'lambda_d' : 1,
              'num_classes' : 3,
              'image_size' : 64,
              'num_primcaps' : 7*3,
              'm_dim' : 5,
              'ad' : 0.6
         }
      else:
          self.option = option

      self.image_size = self.option['image_size']
      self.channels = 3

      fsanet_model_path = fsanet_model_path

      S_set = [self.option['num_capsule'], 
                    self.option['dim_capsule'], 
                    self.option['routings'], 
                    self.option['num_primcaps'], 
                    self.option['m_dim']
                   ]

      model1 = FSA_net_Capsule(self.option['image_size'], self.option['num_classes'], self.option['stage_num'], self.option['lambda_d'], S_set)()

      model2 = FSA_net_Var_Capsule(self.option['image_size'], self.option['num_classes'], self.option['stage_num'], self.option['lambda_d'], S_set)()

      self.option['num_primcaps'] = 8*8*3

      S_set = [self.option['num_capsule'], 
                    self.option['dim_capsule'], 
                    self.option['routings'], 
                    self.option['num_primcaps'], 
                    self.option['m_dim']
                   ]

      model3 = FSA_net_noS_Capsule(self.image_size, self.option['num_classes'], self.option['stage_num'], self.option['lambda_d'], S_set)()

      print('Loading models ...')

      weight_file1 = fsanet_model_path[0]
      model1.load_weights(weight_file1)
      print('Finished loading model 1.')

      weight_file2 = fsanet_model_path[1]
      model2.load_weights(weight_file2)
      print('Finished loading model 2.')

      weight_file3 = fsanet_model_path[2]
      model3.load_weights(weight_file3)
      print('Finished loading model 3.')

      inputs = Input(shape=(self.image_size,self.image_size,self.channels))

      x1 = model1(inputs) #1x1
      x2 = model2(inputs) #var
      x3 = model3(inputs) #w/o

      avg_model = Average()([x1,x2,x3])

      self.model = Model(inputs=inputs, outputs=avg_model)
      
  def getDirectionFromFaces(self,list_faces):

    t = time.time()

    if(len(list_faces)>0):

      results = []

      for face in list_faces:

        info = {}

        face = cv2.resize(face, (64,64))

        face = cv2.normalize(face, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

        face = np.expand_dims(face, axis=0)

        p_result = self.model.predict(face)

        #img = draw_axis(list_faces, p_result[0][0], p_result[0][1], p_result[0][2])

        info['direction'] = p_result
        info['face'] = input_img

        if checkHeadPose(info):
             results.append(info)

        print('head pose estimation time:{0:.2f}'.format(time.time()-t))

    return results
"""  
  def getDirectionFromFaces(self,faces_coordinate,input_img):

      t = time.time()

      results = []  

      if len(faces_coordinate) > 0:

          print(len(faces_coordinate))    

          img_h,img_w,_ = input_img.shape

          faces = np.empty((len(faces_coordinate),self.image_size,self.image_size,self.channels))

          for i, d in enumerate(faces_coordinate):
              info = {}
              x1,y1,w,h = d[0:4]
              x2,y2 = x1+w,y1+h
              x = (x1+x2)//2
              y = (y1+y2)//2
              print(x,y,w,h)
              xw1 = max(int(x - self.option['ad'] * w), 0)
              yw1 = max(int(y - self.option['ad'] * h), 0)
              xw2 = min(int(x + self.option['ad'] * w), img_w - 1)
              yw2 = min(int(y + self.option['ad'] * h), img_h - 1)
              #print(str(xw1)+' '+str(yw1)+' '+str(xw2)+' '+str(yw2))

              print(input_img[yw1:yw2+1, xw1:xw2+1, :].shape)

              faces[i,:,:,:] = cv2.resize(input_img[yw1:yw2+1, xw1:xw2+1, :], (64, 64))
              faces[i,:,:,:] = cv2.normalize(faces[i,:,:,:], None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)        

              face = np.expand_dims(faces[i,:,:,:], axis=0)
              p_result = self.model.predict(face)
              
              img = draw_axis(input_img[yw1:yw2+1, xw1:xw2+1, :], p_result[0][0], p_result[0][1], p_result[0][2])
              
              #print(p_result.shape)
              input_img[yw1:yw2+1, xw1:xw2+1, :] = img

              info['box'] = [x1,y1,w,h]
              info['direction'] = p_result
              info['image'] = input_img
              
              if checkHeadPose(info):
                   results.append(info)
                  
      print('head pose estimation time:{0:.2f}'.format(time.time()-t))

      return results
"""

  def checkHeadPose(self,info):
       
      if (info['direction'][0] >=-HEAD_POSE_THRESHOLD and info['direction'][0] <=HEAD_POSE_THRESHOLD) 
          and (info['direction'][1] >=-HEAD_POSE_THRESHOLD and info['direction'][1] <=HEAD_POSE_THRESHOLD) 
          and (info['direction'][2] >=-HEAD_POSE_THRESHOLD and info['direction'][2] <= HEAD_POSE_THRESHOLD):
          
          return True
      
      return False
      

Overwriting Headpose.py


In [0]:
%%writefile face_direction/FSANET_face_direction.py

import os
import cv2
import sys
import numpy as np
from math import cos, sin
from moviepy.editor import *
def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 80):

    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180
    Pit = "pit: "+str(round(pitch,6))
    Yaw = "yaw: "+str(round(yaw,6))
    Roll = "roll: "+str(round(roll,6))
    #cv2.putText(img, text=Pit, org=(0, 45), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    #                    fontScale=0.50, color=(0, 0, 255), thickness=1)
    #cv2.putText(img, text=Yaw, org=(0, 75), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    #                    fontScale=0.50, color=(0, 255, 0), thickness=1)
    #cv2.putText(img, text=Roll, org=(0, 105), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    #                    fontScale=0.50, color=(255, 0, 0), thickness=1)                                        
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),2)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),2)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img
    
def draw_direction_results(results,input_img,faces,ad,img_size,img_w,img_h,model):
    if len(results) > 0:
        for i, d in enumerate(results):
            y1,x1,y2,x2 = d[1:5]
            x = (x1+x2)*img_w//2
            y = (y1+y2)*img_h//2
            w = h = max((x2-x1)*img_w, (y2-y1)*img_h)
            xw1 = max(int(x - ad * w), 0)
            yw1 = max(int(y - ad * h), 0)
            xw2 = min(int(x + ad * w), img_w - 1)
            yw2 = min(int(y + ad * h), img_h - 1)
            #print(str(xw1)+' '+str(yw1)+' '+str(xw2)+' '+str(yw2))
            faces[i,:,:,:] = cv2.resize(input_img[yw1:yw2+1, xw1:xw2+1, :], (img_size, img_size))
            faces[i,:,:,:] = cv2.normalize(faces[i,:,:,:], None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)        
            
            face = np.expand_dims(faces[i,:,:,:], axis=0)
            p_result = model.predict(face)
            
            face = face.squeeze()
            img = draw_axis(input_img[yw1:yw2+1, xw1:xw2+1, :], p_result[0][0], p_result[0][1], p_result[0][2])
            #print(p_result.shape)
            input_img[yw1:yw2+1, xw1:xw2+1, :] = img

    return input_img 

Overwriting face_direction/FSANET_face_direction.py
